# DSPy Agent Example with Azure OpenAI

In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import dspy
import ujson
import random
import dspy.evaluate
import dspy.retrievers
import dspy.datasets
from datasets import load_dataset

In [28]:
# Load environment variables from .env file
load_dotenv()

True

## Setup Tracing with MLFlow

In [3]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy-Azure-Eval-Example")

# Enable tracing.
mlflow.dspy.autolog()

## Setup LM models

In [32]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

gpt41 = dspy.LM(
    model=f"azure/gpt-4.1",
    base_url=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_ad_token_provider=token_provider,
)

gpt4omini = dspy.LM(
    model=f"azure/gpt-4o-mini",
    base_url=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_ad_token_provider=token_provider,
)

# Use the mini model as the default for DSPy.
dspy.configure(lm=gpt4omini)

In [5]:
gpt41("Say hello to the world!")

['Hello, world! 🌍']

Trace(trace_id=d8216d81f231485a91f23aa62c81f008)

In [33]:
gpt4omini("Say hello to the world!")

['Hello, world! 🌍✨ How can I assist you today?']

Trace(trace_id=4d6a0f7b69e74152860dfa9754ad3592)

## Load HoVer multi-hop QA datasets

In [ ]:
# Load HoVer dataset using parquet files (avoiding deprecated script)
from datasets import load_dataset

try:
    # Try loading using parquet files which should be available
    dataset = load_dataset("parquet", data_files={
        "train": "hf://datasets/hover-nlp/hover/train-00000-of-00001.parquet"
    })
    hover_train = dataset["train"]
except:
    # Fallback: Load using the dataset name with revision
    dataset = load_dataset("hover-nlp/hover", revision="refs/convert/parquet")
    hover_train = dataset["train"]

# Convert to DSPy examples
hover = []
hpqa_ids = set()

for x in hover_train:
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids:
        hpqa_ids.add(x["hpqa_id"])
        titles = list(set([y["key"] for y in x["supporting_facts"]]))
        hover.append(
            dspy.Example(claim=x["claim"], titles=titles).with_inputs("claim")
        )

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

0000.parquet:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/751k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/552k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
len(trainset), len(devset), len(testset)

(100, 100, 1216)

In [9]:
trainset[0]

Example({'claim': 'This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".', 'titles': ['Academy Award for Best Director', 'Chris Noonan', 'Miss Potter']}) (input_keys={'claim'})

## Tools for agent

In [10]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [11]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [12]:
search_wikipedia("Albert Einstein")

['Albert Einstein | Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist. Einstein developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). Einstein\'s work is also known for its influence on the philosophy of science. Einstein is best known by the general public for his mass–energy equivalence formula "E" = "mc" (which has been dubbed "the world\'s most famous equation"). He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the evolution of quantum theory.',
 'Einstein (disambiguation) | Albert Einstein (1879–1955) was a German-born theoretical physicist.',
 'Hans Albert Einstein | Hans Albert Einstein ( ; May 14, 1904 – July 26, 1973) was a Swiss-American engineer and educator, the second child and first son of Albert Einstein and Mileva Marić. Hans A. Einstein was a professor

In [13]:
lookup_wikipedia("Albert Einstein")

'Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist. Einstein developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). Einstein\'s work is also known for its influence on the philosophy of science. Einstein is best known by the general public for his mass–energy equivalence formula "E" = "mc" (which has been dubbed "the world\'s most famous equation"). He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the evolution of quantum theory.'

## Define DSPy agent using `dspy.ReAct`

In [34]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [35]:
react(claim="David Gregory was born in 1625.").titles[:3]

['David Gregory (physician)',
 'David Gregory (historian)',
 'David Gregory (mathematician)']

Trace(trace_id=6894d0db15d248da926ee2233467a561)

## Evaluate the agent

In [36]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [37]:
# Wrapper to bypass exceptions during evaluation.
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 77.00 / 100 (77.0%): 100%|██████████| 100/100 [02:39<00:00,  1.59s/it]

2025/07/13 19:26:10 INFO dspy.evaluate.evaluate: Average Metric: 77.0 / 100 (77.0%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Oxford Movement, Trinity Episcopal Church (Houghton, Michigan), S...","{'thought_0': ""I need to gather information about the Church of En...",The claim states that the Church of England's movement that inspir...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",✔️ [1.000]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Red, White &amp; Crüe, Mike Tyson, Bobby Stewart]",{'thought_0': 'I need to find information about the athlete mentio...,"The claim connects the album ""Red, White & Crüe"" by Mötley Crüe to...","[Bobby Stewart, Mike Tyson, Mötley Crüe, Red, White & Crüe]",✔️ [0.667]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Glen or Glenda, Ed Wood, Fernand Rivers]",{'thought_0': 'I need to verify the claim about the writer/directo...,"The claim states that the writer/director/actor from ""Glen or Glen...","[Glen or Glenda, Ed Wood, Fernand Rivers]",✔️ [1.000]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Chicken Ranch (film), The End of Suburbia, Sandi Sissel]",{'thought_0': 'I need to verify the release date of the film by Sa...,"The claim states that a film by Sandi Sissel was released before ""...","[The End of Suburbia, Sandi Sissel]",✔️ [0.667]
4,The actor who played captain hook in the live production with Tayl...,"[Taylor Louderman, Peter Pan Live!, Christopher Walken]",{'thought_0': 'I need to verify the claim about the actor who play...,The claim states that the actor who played Captain Hook in the liv...,"[Peter Pan Live!, Christopher Walken, The Deer Hunter]",✔️ [0.667]


77.0

[Trace(trace_id=a85ccc04aa744932aadfa42de1f0c6a5), Trace(trace_id=d2af40140ea142e79970ac3ebcaf2e34), Trace(trace_id=3cf3cb53bc274f2cbf4d525e433f1f40), Trace(trace_id=0b6c72f1e4764126ae74f29d5ab67c60), Trace(trace_id=a9c87dc5be7749f8a9126c48f454fa51), Trace(trace_id=d0229d1954614fbba1eb325ab3e4bdf1), Trace(trace_id=24889d9241b4492b925cf49284aaa13f), Trace(trace_id=9cac4eb76e384e36aafafe87bf682a66), Trace(trace_id=ad554d0a4ed14eb783d18b7cef06e1dd), Trace(trace_id=13570e8fa26340ba9af2ed9a1312ab78)]

## Optimize the agent

In [38]:
kwargs = dict(teacher_settings=dict(lm=gpt41), prompt_model=gpt41, max_errors=999)

tp = dspy.MIPROv2(metric=top5_recall, auto="medium", num_threads=16, **kwargs)
optimized_react = tp.compile(react, trainset=trainset, max_bootstrapped_demos=3, max_labeled_demos=0)

2025/07/13 19:27:21 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 28
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 80



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 6 * 2 lm calls in program + (3) lm calls in program-aware proposer = 25 prompt model calls
- Program Evaluation: 35 examples in minibatch * 28 batches + 80 examples in val set * 6 full evals = 1460 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token)
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectati

2025/07/13 19:27:41 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/13 19:27:41 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/13 19:27:41 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...



No input received within 20 seconds. Proceeding with execution...
Bootstrapping set 1/12
Bootstrapping set 2/12


 55%|█████▌    | 11/20 [01:47<01:27,  9.74s/it]


...

2025/07/13 20:10:24 INFO dspy.evaluate.evaluate: Average Metric: 26.0 / 35 (74.3%)


2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 74.29 on minibatch of size 35 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 6', 'Predictor 1: Instruction 5', 'Predictor 1: Few-Shot Set 8'].
2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [65.71, 72.38, 62.86, 67.62, 69.52, 69.52, 77.14, 75.24, 70.48, 77.14, 72.38, 79.05, 75.24, 71.43, 70.48, 77.14, 70.48, 62.86, 68.57, 74.29, 71.43, 67.62, 74.29, 68.57, 70.48, 63.81, 70.48, 74.29]
2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [76.67, 74.58, 72.08, 70.83, 73.33, 72.5]
2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 76.67
2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 35 / 35 - Full Evaluation =====
2025/07/13 20:10:27 INFO dspy.teleprompt.mipro_op

Average Metric: 56.67 / 80 (70.8%): 100%|██████████| 80/80 [02:15<00:00,  1.69s/it]

2025/07/13 20:12:42 INFO dspy.evaluate.evaluate: Average Metric: 56.666666666666664 / 80 (70.8%)
2025/07/13 20:12:42 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [76.67, 74.58, 72.08, 70.83, 73.33, 72.5, 70.83]
2025/07/13 20:12:42 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 76.67
2025/07/13 20:12:42 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/07/13 20:12:42 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/07/13 20:12:42 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 76.67!


[Trace(trace_id=212584daa5d74e2b840700c6875b41da), Trace(trace_id=29418dbd5f394bda83bae53e175eeb0b), Trace(trace_id=4e5a47ac849747c89dc9a883ea14479b), Trace(trace_id=7149b0f09f7e43a49b4664e9de487e9f), Trace(trace_id=aff97184a88b471dbd3c672d0927b1f8), Trace(trace_id=9ccb70a9dbeb40d3a6bfbeddf7f75e9d)]

In [ ]:
optimized_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Joe Orton', 'Bernard-Marie Koltès', 'Up Against It']

Trace(trace_id=5509bcc28b9449538d01b2ad65bf7cdd)

In [40]:
optimized_react.save("optimized_react.json")

loaded_react = dspy.ReAct("claim -> titles: list[str]", tools=[search_wikipedia, lookup_wikipedia], max_iters=20)
loaded_react.load("optimized_react.json")

loaded_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Joe Orton', 'Bernard-Marie Koltès', 'Up Against It']

Trace(trace_id=e63361b54f554208bfe7fbc2007fb61d)